# 附录 10.2: 工具使用

- [课程](#课程)
- [练习](#练习)
- [示例游乐场](#示例游乐场)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:


# 导入Python内置的正则表达式库
import re
import openai

# 从IPython存储中检索API_KEY变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = openai.OpenAI(api_key=API_KEY, base_url=BASE_URL)

# 修改为调用OpenAI GPT-4，并包含stop参数
def get_completion(messages, system_prompt="", prefill="", stop_sequences=None):
    # 如果有system_prompt，将其作为第一条消息
    full_messages = []
    if system_prompt:
        full_messages.append({"role": "system", "content": system_prompt})
    
    # 添加用户消息
    full_messages.extend(messages)
    
    # 如果有prefill，添加为assistant消息开头
    if prefill:
        full_messages.append({"role": "assistant", "content": prefill})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=full_messages,
        max_tokens=2000,
        temperature=0.0,
        stop=stop_sequences
    )
    return response.choices[0].message.content

---

## 课程

虽然一开始可能看起来在概念上很复杂，但工具使用（也称为函数调用）实际上相当简单！您已经掌握了实现工具使用所需的所有技能，这实际上只是替换和提示链接的组合。

在之前的替换练习中，我们将文本替换到提示中。通过工具使用，我们将工具或函数结果替换到提示中。GPT不能直接调用或访问工具和函数。相反，我们让GPT：
1. 输出它想要调用的工具名称和参数
2. 在调用工具时暂停任何进一步的响应生成
3. 然后我们用附加的工具结果重新提示

函数调用很有用，因为它扩展了GPT的能力，使GPT能够处理更复杂的多步骤任务。
您可以为GPT提供的函数示例：
- 计算器
- 字数统计器
- SQL数据库查询和数据检索
- 天气API

您可以通过结合这两个元素让GPT进行工具使用：

1. 系统提示，在其中我们向GPT解释工具使用的概念以及它可以访问的工具的详细描述列表
2. 用于编排和执行GPT工具使用请求的控制逻辑

### 工具使用路线图

*本课程教授当前的工具使用格式。但是，我们将在不久的将来更新和改进工具使用功能，包括：*
* *更简化的函数定义和调用格式*
* *更强大的错误处理和边缘情况覆盖*
* *与API其余部分的更紧密集成*
* *更好的可靠性和性能，特别是对于更复杂的工具使用任务*

### 示例

要在GPT中启用工具使用，我们从系统提示开始。在这个特殊的工具使用系统提示中，我们告诉GPT：
* 工具使用的基本前提及其包含的内容
* GPT如何调用和使用给定的工具
* 在这个特定场景中它可以访问的工具的详细列表

这是系统提示的第一部分，向GPT解释工具使用。系统提示的这一部分在所有提示GPT进行工具使用的实例中都是通用的。我们为GPT提供的工具调用结构（`<function_calls> [...] </function_calls>`）是GPT经过专门训练使用的结构，因此我们建议您坚持使用这种格式。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

这是系统提示的第二部分，它定义了GPT在这个特定情况下可以访问的确切工具。在这个例子中，我们将为GPT提供一个计算器工具，它需要三个参数：两个操作数和一个运算符。

然后我们将系统提示的两个部分结合起来。

In [3]:
system_prompt_tools_specific_tools = """以下是以JSONSchema格式提供的可用函数：
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
用于进行基础算术运算的计算器函数。
支持加法、减法、乘法和除法
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第一个操作数（运算符之前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第二个操作数（运算符之后）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>要执行的运算。必须是 +、-、* 或 / 中的一个</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

现在我们可以给GPT一个需要使用`calculator`工具的问题。我们将在`stop_sequences`中使用`<function_calls\>`来检测GPT是否以及何时调用函数。

In [4]:
multiplication_message = {
    "role": "user",
    "content": "计算 1,984,135 乘以 9,343,116"
}

stop_sequences = ["</function_calls>"]

# 获取GPT的响应
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

<function_calls>
<invoke name="calculator">
<antml:parameter name="first_operand">1984135</antml:parameter>
<antml:parameter name="second_operand">9343116</antml:parameter>
<antml:parameter name="operator">*</antml:parameter>
</invoke>



现在，我们可以从GPT的函数调用中提取参数，并代表GPT实际运行函数。

首先我们定义函数的代码。

In [5]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "错误：不支持该运算。"

然后我们从GPT的函数调用响应中提取参数。如果所有参数都存在，我们就运行计算器工具。

In [6]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

---------------- RESULT ----------------
18,538,003,464,660


现在我们有了结果，我们必须正确格式化该结果，以便当我们将其传回给GPT时，GPT能够理解该结果与哪个工具相关。 GPT经过训练可以识别以下设定格式：
```
<function_results>
<result>
<tool_name>{TOOL_NAME}</tool_name>
<stdout>
{TOOL_RESULT}
</stdout>
</result>
</function_results>
```

运行下面的单元格将上述工具结果格式化为这种结构。

In [7]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

<function_results>
<result>
<tool_name>do_pairwise_arithmetic</tool_name>
<stdout>
18538003464660
</stdout>
</result>
</function_results>


现在我们只需要通过将结果附加到之前相同的消息链中，将此结果发送回GPT，就大功告成了！

In [8]:
full_first_response = function_calling_response + "</function_calls>"

# 构建完整的对话
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# 打印GPT的响应
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

------------- FINAL RESULT -------------
1,984,135 乘以 9,343,116 的结果是 18,538,003,464,660。


恭喜您完成了整个工具使用链的端到端运行！

现在如果我们给GPT一个完全不需要使用给定工具的问题会怎样？

In [9]:
non_multiplication_message = {
    "role": "user",
    "content": "告诉我法国的首都。"
}

stop_sequences = ["</function_calls>"]

# 获取GPT的响应
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

法国的首都是巴黎。


成功！如您所见，GPT知道在不需要时不调用函数。

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例游乐场**](#示例游乐场)。

---

## 练习
- [练习 10.2.1 - SQL](#练习-1021---sql)

### 练习 10.2.1 - SQL
在这个练习中，您将编写一个工具使用提示，用于查询和写入世界上最小的"数据库"。这是初始化的数据库，实际上只是一个字典。

In [10]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

以下是用于向数据库写入和从数据库读取的函数代码。

In [11]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

要解决这个练习，首先要定义一个类似于上面`system_prompt_tools_specific_tools`的系统提示。确保包含每个工具的名称和描述，以及每个函数的每个参数的名称、类型和描述。我们在下面为您提供了一些起始框架。

In [12]:
system_prompt_tools_specific_tools_sql = """
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

准备好后，您可以在下面的示例中试验您的工具定义系统提示。只需运行下面的单元格！

In [13]:
examples = [
    "向数据库添加一个名为Deborah的用户。",
    "向数据库添加一个名为Thingo的产品",
    "告诉我用户2的姓名",
    "告诉我产品3的名称"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # 获取并打印GPT的响应
    function_calling_response = get_completion([message], system_prompt=system_prompt, stop_sequences=stop_sequences)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

向数据库添加一个名为Deborah的用户。 
----------

 <function_calls>
<invoke name="invoke_stored_procedure">
<antml:parameter name="procedure_name">AddUser</antml:parameter>
<antml:parameter name="user_name">Deborah</antml:parameter>
</invoke>
 
*********
*********
*********


向数据库添加一个名为Thingo的产品 
----------

 <function_calls>
<invoke name="invoke_stored_procedure">
<antml:parameter name="procedure_name">AddProduct</antml:parameter>
<antml:parameter name="product_name">Thingo</antml:parameter>
</invoke>
 
*********
*********
*********


告诉我用户2的姓名 
----------

 <function_calls>
<invoke name="get_user_info">
<antml:parameter name="user_id">2</antml:parameter>
</invoke>
 
*********
*********
*********


告诉我产品3的名称 
----------

 <function_calls>
<invoke name="get_product_name">
<antml:parameter name="product_id">3</antml:parameter>
</invoke>
 
*********
*********
*********




如果您做得正确，函数调用消息应该正确调用`add_user`、`add_product`、`get_user`和`get_product`函数。

作为额外的练习，添加一些代码单元格并编写参数解析代码。然后用GPT给您的参数调用函数，查看调用后"数据库"的状态。

❓ 如果您想查看可能的解决方案，请运行下面的单元格！

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### 恭喜！

恭喜您学会了工具使用和函数调用！如果您想了解更多关于搜索和RAG的内容，请前往最后的附录部分。

---

## 示例游乐场

这是一个供您自由试验本课程中显示的提示示例的区域，您可以调整提示以查看它如何影响GPT的响应。

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<nvoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In [ ]:
system_prompt_tools_specific_tools = """以下是以JSONSchema格式提供的可用函数：
<tools>
<tool_description>
<tool_name>calculator</tool_name>
<description>
用于进行基础算术运算的计算器函数。
支持加法、减法、乘法和除法
</description>
<parameters>
<parameter>
<name>first_operand</name>
<type>int</type>
<description>第一个操作数（运算符之前）</description>
</parameter>
<parameter>
<name>second_operand</name>
<type>int</type>
<description>第二个操作数（运算符之后）</description>
</parameter>
<parameter>
<name>operator</name>
<type>str</type>
<description>要执行的运算。必须是 +、-、* 或 / 中的一个</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "计算 1,984,135 乘以 9,343,116"
}

stop_sequences = ["</function_calls>"]

# 获取GPT的响应
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "错误：不支持该运算。"

In [ ]:
def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

first_operand = find_parameter(function_calling_response, "first_operand")
second_operand = find_parameter(function_calling_response, "second_operand")
operator = find_parameter(function_calling_response, "operator")

if first_operand and second_operand and operator:
    result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
    print("---------------- RESULT ----------------")
    print(f"{result:,}")

In [ ]:
def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'do_pairwise_arithmetic',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)

In [ ]:
full_first_response = function_calling_response + "</function_calls>"

# 构建完整的对话
messages = [multiplication_message,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# 打印GPT的响应
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "告诉我法国的首都。"
}

stop_sequences = ["</function_calls>"]

# 获取GPT的响应
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)